# Question and answers with Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

QA(Question Answering)는 자연어로 제기된 사실적 질문에 대한 답변을 추출하는 중요한 작업입니다. 일반적으로 QA 시스템은 구조화되거나 구조화되지 않은 데이터가 포함된 지식 기반에 대한 쿼리를 처리하고 정확한 정보로 응답을 생성합니다. 높은 정확도를 보장하는 것은 특히 엔터프라이즈 사용 사례에서 유용하고 신뢰할 수 있으며 신뢰할 수 있는 질문 응답 시스템을 개발하는 데 핵심입니다.

Titan 및 Claude와 같은 생성 AI 모델은 확률 분포를 사용하여 질문에 대한 응답을 생성합니다. 이 모델은 방대한 양의 텍스트 데이터에 대해 훈련되어 시퀀스에서 다음에 오는 내용이나 특정 단어 뒤에 어떤 단어가 뒤따를지 예측할 수 있습니다. 그러나 이러한 모델은 데이터에 항상 어느 정도의 불확실성이 있기 때문에 모든 질문에 대해 정확하거나 결정적인 답변을 제공할 수 없습니다. 기업은 도메인별 독점 데이터를 쿼리하고 정보를 사용하여 질문에 답해야 하며 일반적으로 모델이 훈련되지 않은 데이터를 사용해야 합니다.

이 모듈에서는 Bedrock Titan 모델을 사용하여 쿼리에 대한 정보 응답을 제공하는 방법을 보여줍니다. 

이 예제에서는 컨텍스트 없이 모델을 실행한 다음 수동으로 컨텍스트를 제공하려고 합니다. 여기서는 `RAG` 확대가 발생하지 않습니다. 이 접근 방식은 짧은 문서 또는 단일 톤 애플리케이션에서 작동하며 모델에 전송된 프롬프트에 모두 맞지 않는 대규모 엔터프라이즈 문서가 있을 수 있는 엔터프라이즈 수준 질문 응답으로 확장되지 않을 수 있습니다. 

### Challenges
- 모델이 질문에 대한 사실적인 답변을 반환하도록 하는 방법

### Proposal
위의 과제에 대해 본 노트북에서는 다음과 같은 전략을 제안합니다.
#### Prepare documents
질문에 답하기 전에 문서를 처리하고 문서 저장소 인덱스에 저장해야 합니다. 
- 여기서는 전체 관련 컨텍스트와 함께 요청을 모델에 보내고 응답을 기대합니다.


## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In [5]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249
# %pip install --quiet langchain==0.0.292

Processing /root/amazon-bedrock-workshop-webinar-kr/dependencies/awscli-*-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/root/amazon-bedrock-workshop-webinar-kr/dependencies/awscli-*-py3-none-any.whl'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "https://bedrock.us-east-1.amazonaws.com"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## Section 1: Q&A with the knowledge of the model
이 섹션에서는 훈련 단계에서 얻은 지식을 기반으로 질문에 답하기 위해 Bedrock 서비스에서 제공하는 모델을 사용하려고 합니다.

이 노트북에서는 Amazon Bedrock 클라이언트의 `Invoke_model()` 메서드를 사용합니다. 이 방법을 사용하는 데 필요한 필수 매개 변수는 Amazon Bedrock 모델 ARN을 나타내는 `modelId`와 작업에 대한 프롬프트인 `body`입니다. 선택한 기본 모델 공급자에 따라 본문 프롬프트가 변경됩니다. 아래에서 자세히 살펴보겠습니다.

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```


## Scenario

모델에게 타이어 교체 정보 제공을 요청하는 상황을 모델링 하려고 합니다. 먼저 교육 데이터를 기반으로 모델에 요청하여 자동차의 특정 제조업체 및 모델에 대한 답변을 제공합니다. 이 기술을 '제로샷'이라고 합니다. 우리는 곧 모델이 우리의 특정 자동차와 관련된 답변을 반환하는 것처럼 보이지만 실제로는 환각을 일으키고 있음을 깨닫게 될 것입니다. 우리가 그것을 알아낼 수 있는 이유는 우리가 가짜 자동차를 통과하고 거의 비슷한 시나리오를 받고 대답하기 때문입니다.

이 상황은 자동차의 특정 제조업체 및 모델에 대한 추가 데이터로 모델의 교육을 보강해야 함을 의미하며, 그러면 모델이 매우 구체적인 답변을 반환합니다. 이 노트북에서는 데이터를 보강하기 위해 외부 소스를 사용하지 않고 RAG 기반 보강 시스템이 작동하는 방식을 시뮬레이션합니다.

최종 테스트를 실행하기 위해 우리는 특정 차량에 대한 타이어 변경 작동 방식을 설명하는 설명서의 전체 세부 섹션을 제공한 다음 모델에서 선별된 응답을 다시 얻기 위해 테스트합니다.

## Task

프로세스를 시작하려면 Bedrock에서 제공하는 모델 중 하나를 선택합니다. 이 사용 사례에서는 Titan을 선택합니다. 이러한 모델은 자동차에 대한 일반적인 질문에 답할 수 있습니다.

예를 들어 Titan 모델에게 Audi의 펑크 난 타이어를 교체하는 방법을 알려달라고 요청합니다.


In [7]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my Audi A8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

#### Let's invoke of the model passing in the JSON body to generate the response

In [8]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

I am unsure


The model is able to give you an answer outlining the process of changing the car flat tire, but the same explanation could be valid for any car. Unfortunately this is not the right answer for an Audi A8, which does not have a spare tire. This is because the model has been trained on data that contains instructions about changing tires on cars.

이 문제의 또 다른 예는 Amazon Tirana와 같이 완전히 가짜인 자동차 브랜드 및 모델에 대해 동일한 질문을 시도하여 볼 수 있습니다.

In [9]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

First, remove the wheel from the bike frame. Then, use the valve extender to remove the tire.
Afterward, insert the valve extender into the new tire and inflate it to the recommended pressure.
Finally, reinstall the wheel and tighten the bolts.


보시다시피 모델이 제공하는 답변은 그럴듯하지만 자동차가 아닌 자전거에 대한 것입니다. 모델은 Amazon Tirana가 자전거라고 가정했습니다. 모델이 환각을 일으키고 있습니다. 

이 문제를 어떻게 해결하고 내 자동차 모델에 유효한 특정 지침에 따라 모델이 답변을 제공하도록 할 수 있습니까? 

2020년 Facebook의 연구에 따르면 LLM 지식은 프롬프트의 일부로 추가 지식 기반을 제공함으로써 즉석에서 보강될 수 있습니다. 

이 접근 방식을 RAG(Retrieval Augmented Generation)라고 합니다. 이를 사용하여 애플리케이션을 개선하는 방법을 살펴보겠습니다. 다음은 Audi A8의 설명서를 발췌한 것입니다(실제로는 실제 설명서가 아니지만 그렇다고 가정해 보겠습니다). 이 문서는 또한 Titan Large의 프롬프트에 완전히 들어갈 만큼 편리하게 짧습니다.

```
Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Rivesla or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible.
```

 
다음으로, 이 텍스트를 가져와 원래 질문과 함께 프롬프트에 '임베드'합니다. 프롬프트는 또한 모델이 컨텍스트로 제공된 정보만 보도록 힌트를 주는 방식으로 작성됩니다.

In [10]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Audi or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

#### 전체 발췌문을 가져와 질문과 함께 모델에게 전달해 보겠습니다.

In [11]:
question = "How can I fix a flat tire on my Audi A8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

##### boto3를 통해 모델을 호출하여 응답을 생성합니다.

In [12]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

To fix a flat tire on your Audi A8, you will need to locate the spare tire, jack, lug wrench, and
owner's manual. These items are typically located in the trunk of your vehicle.

Here are the steps to fix a flat tire on your Audi A8:

1. Locate the spare tire, jack, lug wrench, and owner's manual.
2. Open the trunk of your vehicle.
3. Position the jack near the flat tire under the vehicle.
4. Use the lug wrench to loosen the lug nuts on the flat tire.
5. Raise the vehicle with the jack until the flat tire is off the ground.
6. Remove the lug nuts from the flat tire.
7. Mount the spare tire on the vehicle's hub.
8. Tighten the lug nuts by hand as much as possible.
9. Lower the vehicle back down to the ground with the jack.
10. Use the lug wrench to tighten the lug nuts in a star pattern.
11. Check the tire pressure on the spare tire and adjust as necessary.
12. Put away the tools and close the trunk.

It is important to have your flat tire repaired or replaced as soon as possible to avo

모델이 컨텍스트를 이해하고 관련 답변을 생성하는 데 시간이 걸리므로 몇 초 동안 응답을 기다려야 하므로 사용자 경험이 좋지 않을 수 있습니다.

Bedrock은 또한 모델이 토큰을 생성할 때 서비스가 출력을 생성하는 스트리밍 기능을 지원합니다. 다음은 그렇게 할 수 있는 방법의 예입니다.

In [13]:
from IPython.display import display_markdown,Markdown,clear_output

In [14]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

 To fix a flat tire on your Audi A8, you will need to locate the spare tire, jack, lug wrench, and owner's manual. These items are typically located in the trunk of your vehicle.

Here are the steps to fix a flat tire on your Audi A8:

1. Locate the spare tire, jack, lug wrench, and owner's manual.
2. Open the trunk of your vehicle.
3. Position the jack near the flat tire under the vehicle.
4. Use the lug wrench to loosen the lug nuts on the flat tire.
5. Raise the vehicle with the jack until the flat tire is off the ground.
6. Remove the lug nuts from the flat tire.
7. Mount the spare tire on the vehicle's hub.
8. Tighten the lug nuts by hand as much as possible.
9. Lower the vehicle back down to the ground with the jack.
10. Use the lug wrench to tighten the lug nuts in a star pattern.
11. Check the tire pressure on the spare tire and adjust as necessary.
12. Put away the tools and close the trunk.

It is important to have your flat tire repaired or replaced as soon as possible to avoid further damage to the tire. If you are unable to fix the tire yourself, you can always call for roadside assistance or take your vehicle to a professional tire repair shop.

## Summary

We see the response is a summarized and step by step instruction of how to change the tires . This simple example shows how you can leverage the `RAG` or the Augmentation process to generate a curated response back